# Reading Notes

In [38]:
import numpy as np

### Section 2 - Comparisons on two Examples

Below are implemented all algorithms mentioned in chapter 1. They are all realted to the product function
$$f:\mathbb{R}^n \to \mathbb{R}, \quad f(x_1,...,x_n) \mapsto \prod_{i=1}^n x_i$$
which has derivative
$$g := f^\prime:\mathbb{R}^n \to \mathbb{R^n}, \quad f(x_1,...,x_n) \mapsto \sum_{j=1}^n e_j\prod_{i\neq j}^n x_i.$$


In [37]:
# Evaluation of Product [p.7] 
# note idexing must be shifted back by one since the paper indexes from 1, but python indexes from 0

# setup
x = [1,3,3,2,7]
n = len(x)
x += [None] * n

# START OF ALGORITHM
x[n] = x[0]
for i in range(n+1, 2*n):
    x[i] = x[i - n] * x[i - 1]
y = x[2*n-1]
# END OF ALGORITHM

print(x)
print(y)
print(1*3*3*2*7)
print("yay! the algorithm works!")

[1, 3, 3, 2, 7, 1, 3, 9, 18, 126]
126
126
yay, the algorithm works!


In [49]:
# Forward Differentiation of Product [p.7] 

x = [2,3,5]
n = len(x)
x += [None] * n
dx = [None] * (2 * n) # defining unnecessarily long array so that inexing is more consistent with paper.

def e(i):
    return np.eye(1, n, i)

# START OF ALGORITHM
x[n] = x[0]
dx[n] = e(0)
for i in range(n+1, 2*n):
    x[i] = x[i - n] * x[i - 1]
    dx[i] = x[i - 1] * e(i - n) + x[i - n] * dx[i - 1]
y = x[2*n-1]
g = dx[2*n-1]
# END OF ALGORITHM

print(y)
print(g)
print("yay! algorithm works as expected!")

30
[[15. 10.  6.]]


In [1]:
# Reverse Differentiation of Product [p.8] 

x = [2,3,5]
n = len(x)
x += [None] * n
x_bar = [None] * (2 * n) # defining unnecessarily long array so that inexing is more consistent with paper.

def e(i):
    return np.eye(1, n, i)

# START OF ALGORITHM
x[n] = x[0]
for i in range(n+1, 2*n):
    x[i] = x[i - n] * x[i - 1] # forward step
y = x[2 * n - 1]

x_bar[2 * n - 1] = 1
for i in range(2*n - 1, n, -1):
    x_bar[i - 1] = x_bar[i] * x[i - n] # reverse step
    x_bar[i - n] = x_bar[i] * x[i - 1]
x_bar[0] = x_bar[n]
g = x_bar[0:n]
# END OF ALGORITHM

print("y =", y)
print(g)
print("yay! algorithm works as expected!")

y = 30
[15, 10, 6]
yay! algorithm works as expected!


### Section 3 - Automatic Differentiation on Composite Functions


By using a tree structure we implicitly have a similar memory layout to that of x_1, ..., x_n.

#### Forward Accumulation
The Following node structure has accompanying functions for the forward mode differentiation

In [40]:
import numpy as np

x_Var = None

# Node Classes

# Base Expression
class expression:
    def __str__():
        return "EXPRESSION"

    def diff():
        return "CANT EVALUATE DERIVATIVE OF BASE EXPRESSION CLASS"

# Base Expression Types
    
class Un_Op (expression):
    def __init__(self, a):
        self.a = a
    def __str__(self):
        return "[UNARY OP]"

class Bi_Op (expression):
    def __init__(self, a, b):
        self.a = a
        self.b = b
    def __str__(self):
        return "[BINARY OP]"
    
# Usable Expression Types

# Constants
class Const_Exp(expression):
    def __init__(self, a):
        self.a = a
    def __str__(self):
        return str(self.a)
    
    def diff(self):
        return (self.a, 0)

# Variables  
class X_Exp(expression):
    def __init__(self):
        return
    def __str__(self):
        return "x"

    def diff(self):
        return (x_Var, 1)
    
# Unary Operations
class Sin_Op(Un_Op):
    def __str__(self):
        return f"sin({str(self.a)})"
    def diff(self):
        a_val, a_prime = self.a.diff()
        f_prime = np.cos(a_val) * a_prime
        return (np.sin(a_val), f_prime)
    
class Cos_Op(Un_Op):
    def diff(self):
        a_val, a_prime = self.a.diff()
        f_prime = -np.sin(a_val) * a_prime
        return (np.cos(a_val), f_prime)
    def __str__(self):
        return f"cos({str(self.a)})"
    
class Log_Op(Un_Op):
    def diff(self):
        a_val, a_prime = self.a.diff()
        f_prime = (1/a_val) * a_prime
        return (np.log(a_val), f_prime)   
    def __str__(self):
        return f"ln({str(self.a)})"
    
class Exp_Op(Un_Op):
    def diff(self):
        a_val, a_prime = self.a.diff()
        f_prime = np.exp(a_val) * a_prime
        return (np.exp(a_val), f_prime)
    def __str__(self):
        return f"exp({str(self.a)})"

# Binary Operations
class Add_Op(Bi_Op):
    def diff(self):
        a_val, a_prime = self.a.diff()
        b_val, b_prime = self.b.diff()
        f_prime = a_prime + b_prime
        return (a_val + b_val, f_prime)
    def __str__(self):
        return f"({str(self.a)} + {str(self.b)})"

class Sub_Op(Bi_Op):
    def diff(self):
        a_val, a_prime = self.a.diff()
        b_val, b_prime = self.b.diff()
        f_prime = a_prime - b_prime
        return (a_val - b_val, f_prime)
    def __str__(self):
        return f"({str(self.a)} - {str(self.b)})"   
    
class Mult_Op(Bi_Op):
    def diff(self):
        a_val, a_prime = self.a.diff()
        b_val, b_prime = self.b.diff()
        f_prime = (a_val * b_prime) + (a_prime * b_val)
        return (a_val * b_val, f_prime)
    def __str__(self):
        return f"({str(self.a)} * {str(self.b)})"  
    
class Div_Op(Bi_Op):
    def diff(self):
        a_val, a_prime = self.a.diff()
        b_val, b_prime = self.b.diff()
        f_prime = ((a_prime * b_val) - (a_val * b_prime)) / (b_val * b_val)
        return (a_val / b_val, f_prime)
    def __str__(self):
        return f"({str(self.a)} / {str(self.b)})"
    
func = Log_Op(Div_Op(Const_Exp(28), Mult_Op(Add_Op(Sin_Op(X_Exp()), Const_Exp(5)), Const_Exp(2))))
print("f(x) = " + str(func))
x_Var = 2
print(f"Let x = {x_Var}")

print("(f(x), f'(x)) = ", func.diff())
print("yay! this is the correct (f(x), f'(x)) according to wolframalpha")

f(x) = ln((28 / ((sin(x) + 5) * 2)))
Let x = 2
(f(x), f'(x)) =  (0.8625303839735842, 0.07042238805886022)
yay! this is the correct (f(x), f'(x)) according to wolframalpha


#### Backward Accumulation
The following node structure has accompanying functions for backward mode differentiation

In [41]:
import numpy as np

x_Var = None

# Node Classes

# Base Expression
class expression_b:
    def __str__():
        return "EXPRESSION"

    def diff():
        return "CANT EVALUATE DERIVATIVE OF BASE EXPRESSION CLASS"

# Base Expression Types
    
class Un_Op_b(expression_b):
    def __init__(self, a):
        self.a = a
    def __str__(self):
        return "[UNARY OP]"

class Bi_Op_b(expression_b):
    def __init__(self, a, b):
        self.a = a
        self.b = b
    def __str__(self):
        return "[BINARY OP]"
    
# Usable Expression Types

# Constants
class Const_Exp_b(expression_b):
    def __init__(self, a):
        self.a = a
    def __str__(self):
        return str(self.a)
    
    def diff(self):
        return (self.a, 0)

# Variables  
class X_Exp_b(expression_b):
    def __init__(self):
        return
    def __str__(self):
        return "x"

    def diff(self):
        return (x_Var, 1)
    
# Unary Operations
class Sin_Op_b(Un_Op_b):
    def __str__(self):
        return f"sin({str(self.a)})"
    def diff(self):
        a_val, a_prime = self.a.diff()
        f_prime = np.cos(a_val) * a_prime
        return (np.sin(a_val), f_prime)
    
class Cos_Op_b(Un_Op_b):
    def diff(self):
        a_val, a_prime = self.a.diff()
        f_prime = -np.sin(a_val) * a_prime
        return (np.cos(a_val), f_prime)
    def __str__(self):
        return f"cos({str(self.a)})"
    
class Log_Op_b(Un_Op_b):
    def diff(self):
        a_val, a_prime = self.a.diff()
        f_prime = (1/a_val) * a_prime
        return (np.log(a_val), f_prime)   
    def __str__(self):
        return f"ln({str(self.a)})"
    
class Exp_Op_b(Un_Op_b):
    def diff(self):
        a_val, a_prime = self.a.diff()
        f_prime = np.exp(a_val) * a_prime
        return (np.exp(a_val), f_prime)
    def __str__(self):
        return f"exp({str(self.a)})"

# Binary Operations
class Add_Op_b(Bi_Op_b):
    def diff(self):
        a_val, a_prime = self.a.diff()
        b_val, b_prime = self.b.diff()
        f_prime = a_prime + b_prime
        return (a_val + b_val, f_prime)
    def __str__(self):
        return f"({str(self.a)} + {str(self.b)})"

class Sub_Op_b(Bi_Op_b):
    def diff(self):
        a_val, a_prime = self.a.diff()
        b_val, b_prime = self.b.diff()
        f_prime = a_prime - b_prime
        return (a_val - b_val, f_prime)
    def __str__(self):
        return f"({str(self.a)} - {str(self.b)})"   
    
class Mult_Op_b(Bi_Op_b):
    def diff(self):
        a_val, a_prime = self.a.diff()
        b_val, b_prime = self.b.diff()
        f_prime = (a_val * b_prime) + (a_prime * b_val)
        return (a_val * b_val, f_prime)
    def __str__(self):
        return f"({str(self.a)} * {str(self.b)})"  
    
class Div_Op_b(Bi_Op_b):
    def diff(self):
        a_val, a_prime = self.a.diff()
        b_val, b_prime = self.b.diff()
        f_prime = ((a_prime * b_val) - (a_val * b_prime)) / (b_val * b_val)
        return (a_val / b_val, f_prime)
    def __str__(self):
        return f"({str(self.a)} / {str(self.b)})"
    
func = Log_Op_b(Div_Op_b(Const_Exp_b(28), Mult_Op_b(Add_Op_b(Sin_Op_b(X_Exp_b()), Const_Exp_b(5)), Const_Exp_b(2))))
print("f(x) = " + str(func))
x_Var = 2
print(f"Let x = {x_Var}")

print("(f(x), f'(x)) = ", func.diff())
print("yay! this is the correct (f(x), f'(x)) according to wolframalpha")

f(x) = ln((28 / ((sin(x) + 5) * 2)))
Let x = 2
(f(x), f'(x)) =  (0.8625303839735842, 0.07042238805886022)
yay! this is the correct (f(x), f'(x)) according to wolframalpha
